<a href="https://colab.research.google.com/github/pevu97/Housing-Price-Regression-NN/blob/main/notebooks/Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Import of libraries and data

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

!git clone https://github.com/pevu97/Housing-Price-Regression-NN.git


np.set_printoptions(precision=3, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

fatal: destination path 'Housing-Price-Regression-NN' already exists and is not an empty directory.


'2.18.0'

In [8]:
raw_dataset = pd.read_csv('Housing-Price-Regression-NN/data/housing.csv')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### 2. Data Loading and Initial Exploration

In [9]:
dataset = raw_dataset.copy()
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY


In [10]:
dataset.tail()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20635,-121.090000,39.480000,25.000000,1665.000000,374.000000,845.000000,330.000000,1.560300,78100.000000,INLAND
20636,-121.210000,39.490000,18.000000,697.000000,150.000000,356.000000,114.000000,2.556800,77100.000000,INLAND
20637,-121.220000,39.430000,17.000000,2254.000000,485.000000,1007.000000,433.000000,1.700000,92300.000000,INLAND
20638,-121.320000,39.430000,18.000000,1860.000000,409.000000,741.000000,349.000000,1.867200,84700.000000,INLAND
20639,-121.240000,39.370000,16.000000,2785.000000,616.000000,1387.000000,530.000000,2.388600,89400.000000,INLAND


In [11]:
dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.010029
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


In [12]:
dataset.dropna(inplace=True)
dataset.isnull().sum() / len(dataset)

,0
longitude,0.000000
latitude,0.000000
housing_median_age,0.000000
total_rooms,0.000000
total_bedrooms,0.000000
population,0.000000
households,0.000000
median_income,0.000000
median_house_value,0.000000
ocean_proximity,0.000000


In [13]:
dataset.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [14]:
dataset.describe(include=['object'])

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [15]:
dataset['ocean_proximity'].value_counts()

,count
ocean_proximity,
<1H OCEAN,9034
INLAND,6496
NEAR OCEAN,2628
NEAR BAY,2270
ISLAND,5


In [16]:
px.histogram(dataset, x='median_house_value')

In [17]:
dataset['median_house_value'].value_counts()

,count
median_house_value,
500001.000000,958
137500.000000,119
162500.000000,116
112500.000000,103
187500.000000,92
...,...
321700.000000,1
300800.000000,1
393100.000000,1


In [18]:
index_to_drop = dataset[dataset['median_house_value'] == 500001]
dataset = dataset.drop(index=index_to_drop.index)
px.histogram(dataset, x='median_house_value')

In [19]:
dataset_dummies = pd.get_dummies(dataset, drop_first=True)
dataset_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,False,False,True,False
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,False,False,True,False
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,False,False,True,False
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,False,False,True,False
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,False,False,True,False


### 3. Division into training and test sets

In [20]:
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 15580
test_dataset length: 3895


In [21]:
train_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-122.690000,38.510000,18.000000,3364.000000,501.000000,1442.000000,506.000000,6.685400,313000.000000,False,False,False,False
1859,-124.140000,41.950000,21.000000,2696.000000,578.000000,1208.000000,494.000000,2.275000,122400.000000,False,False,False,True
51,-122.270000,37.820000,43.000000,1868.000000,456.000000,1061.000000,407.000000,1.504500,93800.000000,False,False,True,False
11192,-117.930000,33.820000,28.000000,2444.000000,555.000000,1848.000000,567.000000,3.017900,198800.000000,False,False,False,False
20355,-118.960000,34.190000,16.000000,1807.000000,346.000000,587.000000,296.000000,1.981100,162500.000000,False,False,False,False


In [22]:
test_dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-122.260000,37.850000,52.000000,2202.000000,434.000000,910.000000,402.000000,3.203100,281500.000000,False,False,True,False
13,-122.260000,37.840000,52.000000,696.000000,191.000000,345.000000,174.000000,2.673600,191300.000000,False,False,True,False
19,-122.270000,37.840000,52.000000,1503.000000,298.000000,690.000000,275.000000,2.603300,162900.000000,False,False,True,False
26,-122.280000,37.850000,49.000000,1130.000000,244.000000,607.000000,239.000000,2.459700,93800.000000,False,False,True,False
40,-122.260000,37.830000,52.000000,1665.000000,419.000000,946.000000,395.000000,2.097800,155400.000000,False,False,True,False


In [23]:
px.scatter_matrix(train_dataset, dimensions=['households', 'population', 'total_rooms', 'housing_median_age', 'median_house_value'])

In [24]:
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,15580.000000,-119.558780,2.006237,-124.350000,-121.750000,-118.490000,-117.990000,-114.470000
latitude,15580.000000,35.648614,2.147016,32.550000,33.930000,34.270000,37.730000,41.950000
housing_median_age,15580.000000,28.489217,12.505895,1.000000,18.000000,29.000000,37.000000,52.000000
total_rooms,15580.000000,2620.014506,2195.654212,2.000000,1441.000000,2112.000000,3119.000000,39320.000000
total_bedrooms,15580.000000,539.900578,424.499548,2.000000,299.000000,436.000000,647.000000,6445.000000
population,15580.000000,1441.193068,1160.542775,3.000000,801.000000,1179.000000,1746.000000,35682.000000
households,15580.000000,501.099487,385.039835,2.000000,283.000000,411.000000,605.000000,6082.000000
median_income,15580.000000,3.672427,1.570297,0.499900,2.519975,3.440500,4.581425,15.000100


In [25]:
train_labels = train_dataset.pop('median_house_value')
test_labels = test_dataset.pop('median_house_value')

In [26]:
print(train_labels.shape)
print(test_labels.shape)

(15580,)
(3895,)


### 4. Data Standardization

In [27]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_dataset_scaled = pd.DataFrame(data = scaler.fit_transform(train_dataset), columns=train_dataset.columns)
test_dataset_scaled = pd.DataFrame(data = scaler.transform(test_dataset), columns=test_dataset.columns)

In [28]:
train_dataset_scaled.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-1.560793,1.332770,-0.838769,0.338855,-0.091642,0.000695,0.012728,1.918790,-0.707481,-0.013878,-0.346144,-0.378242
1,-2.283563,2.935044,-0.598874,0.034608,0.089754,-0.200941,-0.018439,-0.889941,-0.707481,-0.013878,-0.346144,2.643812
2,-1.351439,1.011383,1.160353,-0.342512,-0.197652,-0.327610,-0.244397,-1.380629,-0.707481,-0.013878,2.888967,-0.378242
3,0.811884,-0.851727,-0.039120,-0.080168,0.035571,0.350543,0.171158,-0.416831,-0.707481,-0.013878,-0.346144,-0.378242
4,0.298469,-0.679390,-0.998698,-0.370295,-0.456789,-0.736053,-0.532688,-1.077110,-0.707481,-0.013878,-0.346144,-0.378242


In [29]:
train_dataset_scaled.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
ocean_proximity_INLAND,0
ocean_proximity_ISLAND,0


In [30]:
test_dataset_scaled.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
ocean_proximity_INLAND,0
ocean_proximity_ISLAND,0


### 5. Model Construction



In [31]:
def build_model():
  model = Sequential()
  model.add(Dense(1024, input_shape=[len(train_dataset_scaled.columns)], activation='relu', kernel_regularizer='l2'))
  model.add(Dropout(0.2))
  model.add(Dense(512, activation='relu', kernel_regularizer='l2'))
  model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
  model.add(Dense(1))

  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
  return model

In [42]:
model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 1024)           │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',       # na jakiej metryce się skupiamy
    patience=5,               # ile epok poczeka zanim przerwie
    restore_best_weights=True  # wróć do najlepszych wag (super ważne!)
)

history = model.fit(train_dataset_scaled, train_labels, epochs=200, validation_split=0.2, verbose=1, batch_size=32,  callbacks=[early_stop])

Epoch 1/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 31965921280.0000 - mae: 144875.2656 - mse: 31965921280.0000 - val_loss: 4596615168.0000 - val_mae: 49282.7461 - val_mse: 4596615168.0000
Epoch 2/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - loss: 4562934784.0000 - mae: 48159.3242 - mse: 4562934784.0000 - val_loss: 3816242944.0000 - val_mae: 44893.5000 - val_mse: 3816242432.0000
Epoch 3/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 3934466304.0000 - mae: 45208.1836 - mse: 3934466048.0000 - val_loss: 3629263616.0000 - val_mae: 44194.0547 - val_mse: 3629263104.0000
Epoch 4/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 3678819072.0000 - mae: 43209.8203 - mse: 3678819072.0000 - val_loss: 3537934080.0000 - val_mae: 43626.3477 - val_mse: 3537934080.0000
Epoch 5/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - loss: 3404892160.0000 - mae: 41903.5000 - mse: 3404891648.0000 - val_loss: 3444822784.0000 - val_mae: 42496.7578 - val_mse: 3444822784.0000
Epoch 6/200
390

In [44]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

In [45]:
test_predictions = model.predict(test_dataset_scaled).flatten()
test_predictions

122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


array([234174.81, 181211.53, 168682.  , ..., 100688.19, 126222.32,  79256.34], dtype=float32)

In [46]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

,median_house_value,predictions
10,281500.000000,234174.812500
13,191300.000000,181211.531250
19,162900.000000,168682.000000
26,93800.000000,145152.218750
40,155400.000000,164886.218750


In [47]:
fig = px.scatter(pred, x='median_house_value', y='predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000]))
fig.show()

In [48]:
pred.head()

,median_house_value,predictions
10,281500.000000,234174.812500
13,191300.000000,181211.531250
19,162900.000000,168682.000000
26,93800.000000,145152.218750
40,155400.000000,164886.218750


In [49]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

,median_house_value,predictions,error
10,281500.000000,234174.812500,47325.187500
13,191300.000000,181211.531250,10088.468750
19,162900.000000,168682.000000,-5782.000000
26,93800.000000,145152.218750,-51352.218750
40,155400.000000,164886.218750,-9486.218750


In [50]:
px.histogram(pred, x='error')